# Multivariate Time Series Restaurant Demand forecasting

 ## **Business Problem and Limitations -**

 Our dataset is for restaurant sales for Tuesday and Wednesday, both lunch and dinner time.
 
There are few instances of 'To-Go' orders like Uber Eats in this dataset.
 
Typical lunch hour is 11:30 AM-2:00 PM, and dinner hour is 6:30 PM-10:00 PM
 
The data set is just for Tuesday and Wednesday.
We needs to expand and randomize the data for min. of 6 months (Jan. 2019 to June 2019) for all days of the week.

A typical restaurant has high covers (number of customers) on Wednesday, Weekend Dinner, followed by Weekend Lunch, and then relatively low covers for Monday to Friday Lunch.
 
The data expansion/randomization should follow the above pattern for the number of customers.

### **Our Gole -**
Predict the top 'Menu Item' and 'Item Qty' for Lunch and Dinner. 
These predictions need to be for future dates (Monday to Sunday, July 1st to July 7th)

## **Implementation -**

## **1) Data Preparation and Preprocessing**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_row',None)
sns.set_style('darkgrid') 

In [ ]:
df = pd.read_excel('../input/restaurant-dataxlsx/Data.xlsx')
df.head()

In [ ]:
# Dropping unnecessary columns
df = df.drop(['StoreCode','DTS','Month','Date','Year','TicketCode'],axis = 1)
df.head()

In [ ]:
# Cheking data type of all columns
df.info()

In [ ]:
# Replacing null by '0' as there is no peoples when food orderd online.
df['PartySize'] = df['PartySize'].replace(['na'],0)
df.head()

In [ ]:
# Normalizing text 
df['MenuCateogry'] = df['MenuCateogry'].str.capitalize()
df['MenuItem'] = df['MenuItem'].str.capitalize()
df.head()

After removing unnecessary columns and imputing null values of the dataset, as shown above, We are ready for data preparation as per the pattern provided above.

As in our dataset, Wednesday is given as weekend and it's only available weekend data, So we are using it as it is for further data preparation.
Now we are dividing data into two groups first one is a weekday and another one is the weekend.

In [ ]:
# Dividing data based on Weekday
weekday = df[df['Day Type']=='Weekday']
print(weekday.shape)
weekday.head()

In [ ]:
# Dividing data based on Weekend
weekend = df[df['Day Type']=='Weekend']
print(weekend.shape)
weekend.head()

Creating data for each day in a week as per the limitations and information we have.

In [ ]:
# Monday - WeekDay
monday = weekday.copy()
monday['Day'] = weekday['Day'].str.replace('Tuesday','Monday') 

# Tuseday - WeekDay
tuesday = weekday.copy() 

# Wednesday - WeekEnd
wednesday = weekend.copy()

# Thusday - WeekDay
thursday = weekday.copy()                                         
thursday['Day'] = thursday['Day'].str.replace('Tuesday','Thursday')

# Friday - WeekDay
friday = weekday.copy()                                          
friday['Day'] = friday['Day'].str.replace('Tuesday','Friday')

# Saturday - WeekEnd
saturday = weekend.copy()                                        
saturday['Day'] = saturday['Day'].str.replace('Wednesday','Saturday')

# Sunday - WeekEnd
sunday = weekend.copy()                                          
sunday['Day'] = sunday['Day'].str.replace('Wednesday','Sunday')

In [ ]:
# Creating data for one week 
week = []
week = pd.concat([tuesday,wednesday,thursday,friday,saturday,sunday,monday,],axis = 0)
print(week.shape)
week.head()

After creating data for one week now we have to expanding it for 6 months with the same pattern. So we are writing a function for it.

In [ ]:
# Creating data for 6 Months with help of above data
months = week.copy()
x = 0
while x < 25:
    months = pd.concat([months,week],axis = 0)
    x = x+1
months.reset_index(drop=True, inplace=True)
print(months.shape)
months.head(10)

In [ ]:
months.info()

In [ ]:
# Creating dates for dataframe.
o = pd.date_range(start='1/1/2019', periods=(len(months)/100), freq='D')
date = []
for i in o:
    for j in range(100):
        date.append(i)
date = pd.DataFrame(date,columns = ['Date'])
date.head()

In [ ]:
# Concating Dates and Months Dataframe
final = pd.concat([date,months],axis = 1)
final.head()

In [ ]:
# Changing Columns Postions for better understanding
final = final[['Date', 'Shift', 'Day Type', 'Day', 'PartySize', 'MenuCateogry','MenuItem', 'ItemPrice', 'ItemQty']]
final = final.iloc[:18100,:]
final.head()

In [ ]:
final.info()

## Here our Data Preparation is finally completed, we maintain all patterns and consider other constraints so that data matches real-world data.
### Now we use final data for further Process.

In [ ]:
df  = final
df.head()

## **2) Visualization**

In [ ]:
# Extracting Two Columns for visualization purpose.
product_1 = df[['MenuItem','ItemQty']]

# Combining two rows 'MenuItem' and 'ItemQty' for Analysis and multiplying based on ItemQty
product = product_1.loc[product_1.index.repeat(product_1.ItemQty)].reset_index(drop=True)
product = product[['MenuItem']]
product.head()

### 1. Word Cloud

In [ ]:
#pip install WordCloud  # Kindly install WordCloud package for furthur process 
# Joinining all the reviews into single paragraph  
speaker_rev_string1 = " ".join(product['MenuItem'])
from wordcloud import WordCloud
wordcloud_sp = WordCloud(width=6000,height=1800).generate(speaker_rev_string1)
plt.axis("off")
plt.tight_layout(pad=0)
plt.imshow(wordcloud_sp)

### 2. Barplot

In [ ]:
p = pd.DataFrame(product_1.groupby(['MenuItem']).sum())
p = p.reset_index()
p.sort_values(by=['ItemQty'], inplace=True,ascending=False)
plt.figure(figsize=(35,8))
chart = sns.barplot(x="MenuItem", y="ItemQty", data=p)
plt.xticks(rotation=90)

## 3) Model Building
In order to build a model firstly, we are modifying some arrangements of the table to facilitate multivariate forecasting.

In [ ]:
df = df[['Date','Shift','MenuItem','ItemQty']]
df.head()

In [ ]:
# Preparing data to use to make Cross Table
new = df.loc[df.index.repeat(df.ItemQty)]
new = new[['Date','Shift','MenuItem']]
new.head(10)

In [ ]:
# Shifting Table
table = pd.DataFrame(pd.crosstab(new.Date,[new.Shift,new.MenuItem]))
table.head()

In [ ]:
# Normalizing Table Names.
table.columns = table.columns.map('{0[1]}-{0[0]}'.format) 
print(table.shape)
table.head()

Looking Behavior of above Data

In [ ]:

plt.figure(figsize=(5,60))
plt.rcParams["figure.figsize"] = [35, 8]
table.plot(legend = False)
# We Can see that data follows seasonality

**Splitting Data into Train and Test.**

For the time series model we can not split our data randomly we should follow seasonilty pattern of data while splitting. So we are splitting as follows.

In [ ]:
Train = table[:int(0.85*(len(table)))]
Test = table[int(0.85*(len(table))):]
print(Train.shape,Test.shape)

For this data, we are using a Data-driven time series model such as smoothing techniques.

Importing libraries necessary for model building

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing 
from statsmodels.tsa.holtwinters import Holt 
from statsmodels.tsa.holtwinters import ExponentialSmoothing 
import warnings
warnings.filterwarnings('ignore')

**Now we are building different models for each item of the menu and make a data frame of respective RMSE to detect which model is best among all.**

### 1st Model:-
**Winter Exponential Smoothing with Additive Seasonality and Additive Trend Model**

In [ ]:
p = []
for i in table.columns:
    hwe_model_add_add = ExponentialSmoothing(Train[i],seasonal="add",trend="add",seasonal_periods=7).fit()
    pred_hwe_add_add = hwe_model_add_add.predict(start = Test.index[0],end = Test.index[-1])
    rmse_hwe_add_add = np.sqrt(np.mean((Test[i]-pred_hwe_add_add)**2))
    p.append(round(rmse_hwe_add_add,3))
p = pd.DataFrame(p, columns = ['Winter_Exponential_Smoothing_RMSE'])

### 2nd Model:- 
**Holt Method Model**

In [ ]:
q = []
for j in table.columns:
    hw_model = Holt(Train[j]).fit()
    pred_hw = hw_model.predict(start = Test.index[0],end = Test.index[-1])
    rmse_hw = np.sqrt(np.mean((Test[j]-pred_hw)**2))
    q.append(round(rmse_hw,3)) 
p['Holt method Model_RMSE']= pd.DataFrame(q, columns = ['Holt method Model_RMSE'])

### 3rd Model:- 
**Simple Exponential Mode**

In [ ]:
r = []
for o in table.columns:
    ses_model = SimpleExpSmoothing(Train[o]).fit()
    pred_ses = ses_model.predict(start = Test.index[0],end = Test.index[-1])
    rmse_ses = np.sqrt(np.mean((Test[o]-pred_ses)**2))
    r.append(round(rmse_ses,3)) # 0.49
p['Simple Exponential Mode_RMSE']= pd.DataFrame(r, columns = ['Simple Exponential Mode_RMSE'])

RMSE Data Frame of each model for each item.

In [ ]:
p.head()

Sum of RMSE's model wise.

In [ ]:
p.sum()

### **From above we can see that Winters Exponential Smoothing with a seasonality of 7 showing less RMSE values as compare to others, So we are selecting it as the final model for our prediction.**

## 4) Building Functions
We are building two different functions first one is to forecast the demand for all items in the menu and the second one is to forecast the demand of the top N item on the menu.


### 1st Function:-
Forecast the demand for all items and normalize the arrangement of Dataframe in Lunch and dinner formate.

**Input parameters-**

***table*** - Historical Dataframe after modification.

***start_date*** - First date of the period which we want to predict.

**end_date** - Last date of the period which we want to predict.

In [ ]:
def Daily_menu_forcasting(table,start_date,end_date):
    da = pd.date_range(start = start_date, end = end_date , freq='D')
    for_pred = pd.DataFrame(da,columns = ['Date'] )
    for_pred = for_pred.set_index('Date')
    for i in table.columns:
        hwe_model_add_add = ExponentialSmoothing(table[i],seasonal="add",trend="add",seasonal_periods=7).fit()
        pred_hwe_add_add = hwe_model_add_add.predict(start = for_pred.index[0],end = for_pred.index[-1])
        for_pred[i]=((round(pred_hwe_add_add)).astype(int))
    final_pred =  for_pred
    p = pd.DataFrame(final_pred.stack())
    p = p.reset_index()
    p[['MenuItem','Shift']] = p.level_1.str.split("-",expand=True,)
    p = p.rename(columns={0: "ItemQty"})
    p = p[['Date','Shift','MenuItem',"ItemQty"]]
    p = p[p['ItemQty'] != 0]
    # Makind Dataframe with dinner and lunch columns
    new = p.loc[p.index.repeat(p.ItemQty)]
    f = pd.DataFrame(pd.crosstab([new.Date,new.MenuItem],[new.Shift]))
    f = f.reset_index()

    # Shorting Data Frame on the basis top item
    f['Total orders of Day'] = f.Dinner + f.Lunch
    f = f.sort_values(['Date', 'Total orders of Day'], ascending=[True, False]).reset_index(drop= True)
    f
    Daily_req_FiNal_Ans = f.copy()
    return Daily_req_FiNal_Ans

### 2nd Function:-
Forecast the demand for top N items and normalize the arrangement of Dataframe in Lunch and dinner formate.

**Input parameters-**

***table*** - Historical Dataframe after modification.

***start_date*** - First date of the period which we want to predict.

**end_date** - Last date of the period which we want to predict.

**N** - Number of the top items we want. (default n=5)

In [ ]:
def Daily_top_menu_forcasting(table,start_date,end_date,N=5):
    da = pd.date_range(start = start_date, end = end_date , freq='D')
    for_pred = pd.DataFrame(da,columns = ['Date'] )
    for_pred = for_pred.set_index('Date')
    for i in table.columns:
        hwe_model_add_add = ExponentialSmoothing(table[i],seasonal="add",trend="add",seasonal_periods=7).fit()
        pred_hwe_add_add = hwe_model_add_add.predict(start = for_pred.index[0],end = for_pred.index[-1])
        for_pred[i]=((round(pred_hwe_add_add)).astype(int))
    final_pred =  for_pred
    p = pd.DataFrame(final_pred.stack())
    p = p.reset_index()
    p[['MenuItem','Shift']] = p.level_1.str.split("-",expand=True,)
    p = p.rename(columns={0: "ItemQty"})
    p = p[['Date','Shift','MenuItem',"ItemQty"]]
    p = p[p['ItemQty'] != 0]
    # Makind Dataframe with dinner and lunch columns
    new = p.loc[p.index.repeat(p.ItemQty)]
    f = pd.DataFrame(pd.crosstab([new.Date,new.MenuItem],[new.Shift]))
    f = f.reset_index()

    # Shorting Data Frame on the basis top item
    f['Total orders of Day'] = f.Dinner + f.Lunch
    f = f.sort_values(['Date', 'Total orders of Day'], ascending=[True, False]).reset_index(drop= True)
    f
    # Finding Topr product for days.
    name =((f['Date'].astype(str)).unique()).tolist()
    t = pd.DataFrame(columns = f.columns)
    for i in name:
        v = pd.DataFrame((f[f['Date']==i]).head(N))
        t = pd.concat([t,v],axis = 0)
    Daily_top_FiNal_Ans = t.reset_index(drop = True)
    return(Daily_top_FiNal_Ans)

Now final Demand forecasting of items on the menu for future dates (Monday to Sunday, July 1st to July 7th) is.

### For total items.

In [ ]:
all_menu = Daily_menu_forcasting(table,'7/1/2019','7/7/2019')
all_menu.head(10) # top manu day wise

### For top N items.

In [ ]:
# Here N = 8
top_8_menu = Daily_top_menu_forcasting(table,'7/1/2019','7/7/2019',8)
top_8_menu.head(10)

## Conclusion -
We successfully forecast the demand for restaurant food items on the menu for Lunch and dinner for a given time period and also identify top items on the menu. This article shows one of many ways of demand forecasting implementation, according to me it is one of the best ways of handling multivariate forecasting business problems.

### ***Please upvote if you find this Notbook is helpful and thank you so much for giving your valuable time.***
for more information about multivariate time series  and data science please visit
https://medium.com/@nilaydeshmukh

My Linkedin - https://www.linkedin.com/in/nilaydeshmukh/